In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble._forest import _generate_unsampled_indices
from sklearn.inspection import permutation_importance
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math

FEATURE IMPORTANCE

(To run the heatmap section it is still required to run the following definitions - up until the loop starts)

In [4]:
#Remove nul rows
def drop_nan(df):
    list_of_nan = []
    for i, row in df.iterrows():
        if pd.isnull(row["bloc"]):
            #print(i)
            list_of_nan.append(i)
            #print(list_of_nan)
    df = df.drop(list_of_nan)
    return df

In [ ]:
#Import data
df_train = pd.read_csv('../Fresh_start/rl_train_set_scaled_rewarded_abchange.csv')
df_train = drop_nan(df_train)
df_val = pd.read_csv('../Fresh_start/rl_val_set_scaled_rewarded_abchange.csv')
df_val = drop_nan(df_val)
df_test = pd.read_csv('../Fresh_start/rl_test_set_scaled_rewarded_abchange.csv')
df_test = drop_nan(df_test)
df_orig = pd.read_csv('../Matlab/MIMIC_mimiciii.csv')
df_orig = drop_nan(df_orig)

In [8]:
#Complete mortality at 90d column
for i, row in df_orig.iterrows():
    if pd.isnull(row["mortality_90d"]):
        df_orig.at[i, "mortality_90d"] = 0

In [9]:
print(len(df_orig.index), df_orig['icustayid'].nunique(), df_train['icustayid'].nunique(), df_test['icustayid'].nunique(), df_val['icustayid'].nunique())

278578 20956 14669 4192 2095


In [10]:
#df_train.head()

,index,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,...,output_4hourly,cumulated_balance,SOFA,SIRS,vaso_input,iv_input,action,reward,max_dose_vaso_unnorm,abchange_vc
0,0,0.000000,3,7245486000,0.0,0.412268,0.0,0.0,0.0,NaN,...,0.728838,0.382109,0.26087,0.50,0.0,2.0,10,1.303074,0.0,0.0
1,1,0.222560,3,7245500400,0.0,0.412268,0.0,0.0,0.0,NaN,...,0.698697,0.381732,0.26087,0.50,0.0,2.0,10,-0.025000,0.0,0.0
2,2,0.356608,3,7245514800,0.0,0.412268,0.0,0.0,0.0,NaN,...,0.676123,0.381457,0.26087,0.50,0.0,2.0,10,-0.025000,0.0,0.0
3,3,0.452837,3,7245529200,0.0,0.412268,0.0,0.0,0.0,NaN,...,0.692656,0.381110,0.26087,0.25,0.0,2.0,10,-0.025000,0.0,0.0
4,4,0.527957,3,7245543600,0.0,0.412268,0.0,0.0,0.0,NaN,...,0.732052,0.380524,0.26087,0.50,0.0,2.0,10,-0.025000,0.0,0.0


In [ ]:
#Create action map
inv_action_map = {}
count = 0
for i in range(5):
    for j in range(5):
        inv_action_map[count] = [i,j]
        count += 1

In [ ]:
# Columns of interest for feature importance, clinician
state_features_vc_clinician = ['gender','mechvent','re_admission', 'age','Weight_kg','GCS','HR','SysBP','MeanBP',
                  'DiaBP','RR','Temp_C','FiO2_1','Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium','Hb',
                  'WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2','Arterial_BE','HCO3',
                  'Arterial_lactate','SOFA','SIRS','Shock_Index','PaO2_FiO2','cumulated_balance', 'SpO2','BUN',
                  'Creatinine','SGOT','SGPT','Total_bili','INR','input_total','input_4hourly','output_total', 
                  'output_4hourly', 'vaso_input']

In [ ]:
# Columns of interest for feature importance, RL model
state_features_vc_ai = ['gender','mechvent','re_admission', 'age','Weight_kg','GCS','HR','SysBP','MeanBP',
                  'DiaBP','RR','Temp_C','FiO2_1','Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium','Hb',
                  'WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2','Arterial_BE','HCO3',
                  'Arterial_lactate','SOFA','SIRS','Shock_Index','PaO2_FiO2','cumulated_balance', 'SpO2','BUN',
                  'Creatinine','SGOT','SGPT','Total_bili','INR','input_total','input_4hourly','output_total', 
                  'output_4hourly', 'agent_vaso']

In [ ]:
def too_large(df):
    df = df.astype(np.float32)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.fillna(0)
    return df

In [ ]:
def oob_classifier_accuracy(clf, X_train, y_train):
    """
    Compute out-of-bag (OOB) accuracy for a scikit-learn random forest
    classifier. We learned the guts of scikit's RF from the BSD licensed
    code:
    https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/ensemble/forest.py#L425
    """
    X = X_train.values
    y = y_train.values

    n_samples = len(X)
    n_classes = len(np.unique(y))
    predictions = np.zeros((n_samples, n_classes))
    for tree in clf.estimators_:
        unsampled_indices = _generate_unsampled_indices(tree.random_state, n_samples, n_samples)
        tree_preds = tree.predict_proba(X[unsampled_indices, :])
        predictions[unsampled_indices] += tree_preds

    predicted_class_indexes = np.argmax(predictions, axis=1)
    predicted_classes = [clf.classes_[i] for i in predicted_class_indexes]

    oob_score = np.mean(y == predicted_classes)
    return oob_score

In [ ]:
def oob_classifier_accuracy1(clf1, X_train1, y_train1):
    """
    Compute out-of-bag (OOB) accuracy for a scikit-learn random forest
    classifier. We learned the guts of scikit's RF from the BSD licensed
    code:
    https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/ensemble/forest.py#L425
    """
    X = X_train1.values
    y = y_train1.values

    n_samples = len(X)
    n_classes = len(np.unique(y))
    predictions = np.zeros((n_samples, n_classes))
    for tree in clf1.estimators_:
        unsampled_indices = _generate_unsampled_indices(tree.random_state, n_samples, n_samples)
        tree_preds = tree.predict_proba(X[unsampled_indices, :])
        predictions[unsampled_indices] += tree_preds

    predicted_class_indexes = np.argmax(predictions, axis=1)
    predicted_classes = [clf1.classes_[i] for i in predicted_class_indexes]

    oob_score = np.mean(y == predicted_classes)
    return oob_score

In [6]:
#This will take a long time. Results are saved so that you only need to run this once.
for i in range (1,11):
    
    #Import model actions
    deeprl2_testactions = pickle.load(open("../vasochange4_dqn_normal_"+i+"/dqn_normal_actions_test.p", "rb"  ))
    deeprl2_valactions = pickle.load(open( "../vasochange4_dqn_normal_"+i+"/dqn_normal_actions_val.p", "rb" ))
    deeprl2_trainactions = pickle.load(open( "../vasochange4_dqn_normal_"+i+"/dqn_normal_actions_train.p", "rb" ))
    
    #Map actions into action map
    test_actions_arr = np.array([inv_action_map[i] for i in deeprl2_testactions])
    val_actions_arr = np.array([inv_action_map[i] for i in deeprl2_valactions])
    train_actions_arr = np.array([inv_action_map[i] for i in deeprl2_trainactions])
    
    #Split into IV fluid actions and vasopressor actions
    df_test['agent_iv'] = test_actions_arr[:,0]
    df_test['agent_vaso'] = test_actions_arr[:,1]

    df_val['agent_iv'] = val_actions_arr[:,0]
    df_val['agent_vaso'] = val_actions_arr[:,1]

    df_train['agent_iv'] = train_actions_arr[:,0]
    df_train['agent_vaso'] = train_actions_arr[:,1]
    
    #Concatonate train, test and val
    rl_data = pd.concat([df_train, df_test],ignore_index=True, axis=0)
    rl_data = pd.concat([rl_data, df_val], ignore_index=True, axis =0)
    rl_data.shape
    
    #Extract relevant columns
    df_clinician = rl_data[state_features_vc_clinician]
    df_ai = rl_data[state_features_vc_ai]
    
    labels_clinician = np.array(df_clinician['vaso_input'])
    labels_clinician[labels_clinician > 0] = 1 
    df_clinician['vaso_input'] = labels_clinician

    labels_ai = np.array(df_ai['agent_vaso'])
    labels_ai[labels_ai > 0] = 1 
    df_ai['agent_vaso'] = labels_ai
    
    y_train = df_clinician['vaso_input']
    X_train = df_clinician.drop('vaso_input', axis = 1)
    y_train = too_large(y_train)
    X_train = too_large(X_train)
    
    y_train1 = df_ai['agent_vaso']
    X_train1 = df_ai.drop('agent_vaso', axis = 1)
    y_train1 = too_large(y_train1)
    X_train1 = too_large(X_train1)
    
    #Clinicians
    if i == 1:
        clf = RandomForestClassifier(n_estimators=100,  n_jobs=-1, oob_score = True, bootstrap = True, random_state=42)
        clf.fit(X_train,y_train)
        
        feat_importances = pd.Series(clf.feature_importances_, index=X_train.columns)
        feat_importances = feat_importances.sort_values(ascending=True)
        feat_importances.to_csv("../feature_import/clinicians_ft.csv")
    
    #RL model
    clf1 = RandomForestClassifier(n_estimators=100,  n_jobs=-1, oob_score = True, bootstrap = True, random_state=42)
    clf1.fit(X_train1,y_train1)
    
    feat_importances1 = pd.Series(clf1.feature_importances_, index=X_train1.columns)
    feat_importances1 = feat_importances1.sort_values(ascending=True)
    feat_importances.to_csv("../feature_import/ai_ft_"+i+".csv")

In [ ]:
model_results_1 = pd.read_csv("../feature_import/ai_ft_1.csv").sort_values("Feature")
model_results_2 = pd.read_csv("../feature_import/ai_ft_2.csv").sort_values("Feature")
model_results_3 = pd.read_csv("../feature_import/ai_ft_3.csv").sort_values("Feature")
model_results_4 = pd.read_csv("../feature_import/ai_ft_4.csv").sort_values("Feature")
model_results_5 = pd.read_csv("../feature_import/ai_ft_5.csv").sort_values("Feature")
model_results_6 = pd.read_csv("../feature_import/ai_ft_6.csv").sort_values("Feature")
model_results_7 = pd.read_csv("../feature_import/ai_ft_7.csv").sort_values("Feature")
model_results_8 = pd.read_csv("../feature_import/ai_ft_8.csv").sort_values("Feature")
model_results_9 = pd.read_csv("../feature_import/ai_ft_9.csv").sort_values("Feature")
model_results_10 = pd.read_csv("../feature_import/ai_ft_10.csv").sort_values("Feature")
original_decisions = pd.read_csv("../feature_import/clinicians_ft.csv")

In [ ]:
model_results_all = pd.DataFrame(model_results_1, columns = ["Feature"])
model_results_all["Score_1"] = model_results_1["Score"]
model_results_all["Score_2"] = model_results_2["Score"]
model_results_all["Score_3"] = model_results_3["Score"]
model_results_all["Score_4"] = model_results_4["Score"]
model_results_all["Score_5"] = model_results_5["Score"]
model_results_all["Score_6"] = model_results_6["Score"]
model_results_all["Score_7"] = model_results_7["Score"]
model_results_all["Score_8"] = model_results_8["Score"]
model_results_all["Score_9"] = model_results_9["Score"]
model_results_all["Score_10"] = model_results_10["Score"]

In [ ]:
model_results_all['Average_Score'] = model_results_all.mean(numeric_only=True, axis=1)
model_results_all['Max_Score'] = model_results_all.max(numeric_only=True, axis=1)
model_results_all['Min_Score'] = model_results_all.min(numeric_only=True, axis=1)
model_results_all = model_results_all.sort_values("Average_Score", ascending=True)

In [ ]:
model_results_all = (model_results_all.assign(yerr_min = model_results_all.Average_Score-model_results_all.Min_Score)
        .assign(yerr_max=model_results_all.Max_Score-model_results_all.Average_Score))

In [ ]:
original_decisions = original_decisions.rename(columns={'Unnamed: 0': 'Feature', '0': 'Score'}).sort_values("Score", ascending=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 9))
model_results_all.plot(kind="barh", y= "Average_Score", x="Feature", xerr=model_results_all[['yerr_min', 'yerr_max']].T.values, capsize=3, ax=ax2)
original_decisions.plot(kind="barh", y= "Score", x="Feature", ax=ax1)
ax2.set_title("The Model")
ax1.set_title("Original Data")
fig.tight_layout()
plt.show()

HEATMAP

In [ ]:
df_train = pd.read_csv('../Fresh_start/rl_train_set_scaled_rewarded_abchange.csv')
df_train = drop_nan(df_train)
df_preg = pd.read_csv('../Fresh_start/rl_pregnant_set_scaled_rewarded_abchange.csv')
df_preg = drop_nan(df_preg)
df_cont = pd.read_csv('../Fresh_start/rl_control_1_set_scaled_rewarded_abchange.csv')
df_cont = drop_nan(df_cont)
df_test = pd.read_csv('../Fresh_start/rl_test_set_scaled_rewarded_abchange.csv')
df_test = drop_nan(df_test)

In [ ]:
df_train_orig = df_orig[df_orig['icustayid'].isin(df_train['icustayid'].unique())]
df_preg_orig = df_orig[df_orig['icustayid'].isin(df_preg['icustayid'].unique())]
df_cont_orig = df_orig[df_orig['icustayid'].isin(df_cont['icustayid'].unique())]
df_test_orig = df_orig[df_orig['icustayid'].isin(df_test['icustayid'].unique())]

In [ ]:
interventions = df_train_orig[["max_dose_vaso", "input_4hourly"]]

In [ ]:
adjusted_vaso = interventions["max_dose_vaso"][interventions["max_dose_vaso"] >0]
adjusted_iv = interventions["input_4hourly"][interventions["input_4hourly"]>0]

In [ ]:
vaso_vals = [0]
vaso_vals.extend(adjusted_vaso.quantile([0.125,0.375,0.625,0.875]))
iv_vals = [0]
iv_vals.extend(adjusted_iv.quantile([0.125,0.375,0.625,0.875]))

In [ ]:
def sliding_mean(data_array, window=1):
    new_list = []
    for i in range(len(data_array)):
        indices = range(max(i - window + 1, 0),
                        min(i + window + 1, len(data_array)))
        avg = 0
        for j in indices:
            avg += data_array[j]
        avg /= float(len(indices))
        new_list.append(avg)     
    return np.array(new_list)

In [ ]:
phys_actions_preg = pickle.load(open( "../sarsa_phys/phys_actions_preg.p", "rb" ))
phys_actions_cont = pickle.load(open( "../sarsa_phys/phys_actions_cont.p", "rb" ))
phys_actions_test = pickle.load(open( "../sarsa_phys_2/phys_actions_test.p", "rb" ))
phys_actions_train = pickle.load(open( "../sarsa_phys_2/phys_actions_train.p", "rb" ))

In [ ]:
deeprl2_actions_preg = pickle.load(open( "../vasochange4_dqn_normal/dqn_normal_actions_preg.p", "rb" ))
deeprl2_actions_cont = pickle.load(open( "../vasochange4_dqn_normal/dqn_normal_actions_cont.p", "rb" ))
deeprl2_actions_test = pickle.load(open( "../vasochange4_dqn_normal/dqn_normal_actions_test.p", "rb" ))
deeprl2_actions_train = pickle.load(open( "../vasochange4_dqn_normal/dqn_normal_actions_train.p", "rb" ))

In [ ]:
def create_df_counter(actions_list):
    numb = len(actions_list)
    possible_actions = [i for i in range(0,25)]
    chosen_actions = []
    for i in range(0,25):
        chosen_actions.append(actions_list.count(i)/numb)
    df = pd.DataFrame(columns = ["actions","chosen"])
    df["actions"] = possible_actions
    df["chosen"] = chosen_actions
    return df

In [ ]:
deeprl2_df_preg = create_df_counter(deeprl2_actions_preg)
deeprl2_df_cont = create_df_counter(deeprl2_actions_cont)
phys_df_preg = create_df_counter(phys_actions_preg)
phys_df_cont = create_df_counter(phys_actions_cont)

In [ ]:
deeprl2_df = pd.DataFrame(columns = ["actions","chosen_preg","chosen_cont","chosen_cont_sec","diff_preg_cont", "diff_preg_cont_sec"])
deeprl2_df["actions"] = deeprl2_df_preg["actions"]
deeprl2_df["chosen_preg"] = deeprl2_df_preg["chosen"]
deeprl2_df["chosen_cont"] = deeprl2_df_cont["chosen"]
deeprl2_df["chosen_cont_sec"] = deeprl2_df_cont_sec["chosen"]
deeprl2_df["diff_preg_cont"] = deeprl2_df_cont["chosen"] - deeprl2_df_preg["chosen"]
deeprl2_df["diff_preg_cont_sec"] = deeprl2_df_cont_sec["chosen"] - deeprl2_df_preg["chosen"]
deeprl2_df["chosen_preg_prob"] = deeprl2_df_preg["chosen"]*100/sum(deeprl2_df_preg["chosen"])
deeprl2_df["chosen_cont_prob"] = deeprl2_df_cont["chosen"]*100/sum(deeprl2_df_cont["chosen"])
deeprl2_df["chosen_cont_sec_prob"] = deeprl2_df_cont_sec["chosen"]*100/sum(deeprl2_df_cont_sec["chosen"])

In [ ]:
phys_df = pd.DataFrame(columns = ["actions","chosen_preg","chosen_cont","chosen_cont_sec","diff_preg_cont", "diff_preg_cont_sec"])
phys_df["actions"] = phys_df_preg["actions"]
phys_df["chosen_preg"] = phys_df_preg["chosen"]
phys_df["chosen_cont"] = phys_df_cont["chosen"]
phys_df["chosen_cont_sec"] = phys_df_cont_sec["chosen"]
phys_df["diff_preg_cont"] = phys_df_cont["chosen"] - phys_df_preg["chosen"]
phys_df["diff_preg_cont_sec"] = phys_df_cont_sec["chosen"] - phys_df_preg["chosen"]
phys_df["chosen_preg_prob"] = phys_df_preg["chosen"]*100/sum(phys_df_preg["chosen"])
phys_df["chosen_cont_prob"] = phys_df_cont["chosen"]*100/sum(phys_df_cont["chosen"])
phys_df["chosen_cont_sec_prob"] = phys_df_cont_sec["chosen"]*100/sum(phys_df_cont_sec["chosen"])

In [ ]:
x = np.arange(len(phys_df["actions"]))
width = 0.35  # the width of the bars

fig, axp = plt.subplots()
rects1 = axp.bar(x - width, phys_df["chosen_preg_prob"], width*2/3, label='Pregnant')
rects2 = axp.bar(x - width/3, phys_df["chosen_cont_prob"], width*2/3, label='Control 1')
rects3 = axp.bar(x + width/3, phys_df["chosen_cont_sec_prob"], width*2/3, label='Control 2')

# Add some text for labels, title and custom x-axis tick labels, etc.
axp.set_ylabel('Percentage chance??')
axp.set_title('Actions')
#axp.set_xticks(x, phys_df["actions"])
axp.legend()

#axp.bar_label(rects1, padding=3)
#axp.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
x = np.arange(len(deeprl2_df["actions"]))
width = 0.35  # the width of the bars

fig, axp = plt.subplots()
rects1 = axp.bar(x - width, deeprl2_df["chosen_preg_prob"], width*2/3, label='Pregnant')
rects2 = axp.bar(x - width/3, deeprl2_df["chosen_cont_prob"], width*2/3, label='Control 1')
rects3 = axp.bar(x + width/3, deeprl2_df["chosen_cont_sec_prob"], width*2/3, label='Control 2')

# Add some text for labels, title and custom x-axis tick labels, etc.
axp.set_ylabel('Percentage chance??')
axp.set_title('Actions')
#axp.set_xticks(x, phys_df["actions"])
axp.legend()

#axp.bar_label(rects1, padding=3)
#axp.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
phys_actions_tuple_preg = [0 for i in range(len(phys_actions_preg))]
deeprl2_actions_tuple_preg = [0 for i in range(len(deeprl2_actions_preg))]                                      

for i in range(len(phys_actions_preg)):
    #print(deeprl2_actions_preg[i])
    phys_actions_tuple_preg[i] = inv_action_map[phys_actions_preg[i]]
    deeprl2_actions_tuple_preg[i] = inv_action_map[int(deeprl2_actions_preg[i])]

In [ ]:
phys_actions_tuple_cont = [None for i in range(len(phys_actions_cont))]
deeprl2_actions_tuple_cont = [None for i in range(len(phys_actions_cont))]                                      

for i in range(len(phys_actions_cont)):
    phys_actions_tuple_cont[i] = inv_action_map[phys_actions_cont[i]]
    deeprl2_actions_tuple_cont[i] = inv_action_map[deeprl2_actions_cont[i]]

In [ ]:
phys_actions_tuple_test = [0 for i in range(len(phys_actions_test))]
deeprl2_actions_tuple_test = [0 for i in range(len(deeprl2_actions_test))]                                      

for i in range(len(deeprl2_actions_test)): #len(phys_actions_test)
    #print(deeprl2_actions_preg[i])
    phys_actions_tuple_test[i] = inv_action_map[phys_actions_test[i]]
    deeprl2_actions_tuple_test[i] = inv_action_map[int(deeprl2_actions_test[i])]

In [ ]:
phys_actions_tuple_train = [0 for i in range(len(phys_actions_train))]
deeprl2_actions_tuple_train = [0 for i in range(len(deeprl2_actions_train))]                                      

for i in range(len(deeprl2_actions_train)): #len(phys_actions_train)
    #print(deeprl2_actions_preg[i])
    phys_actions_tuple_train[i] = inv_action_map[phys_actions_train[i]]
    deeprl2_actions_tuple_train[i] = inv_action_map[int(deeprl2_actions_train[i])]

In [ ]:
phys_actions_tuple_preg = np.array(phys_actions_tuple_preg)
deeprl2_actions_tuple_preg = np.array(deeprl2_actions_tuple_preg)

phys_actions_tuple_cont = np.array(phys_actions_tuple_cont)
deeprl2_actions_tuple_cont = np.array(deeprl2_actions_tuple_cont)

phys_actions_tuple_test = np.array(phys_actions_tuple_test)
deeprl2_actions_tuple_test = np.array(deeprl2_actions_tuple_test)

phys_actions_tuple_train = np.array(phys_actions_tuple_train)
deeprl2_actions_tuple_train = np.array(deeprl2_actions_tuple_train)

In [ ]:
phys_actions_iv_preg = phys_actions_tuple_preg[:,0]
phys_actions_vaso_preg = phys_actions_tuple_preg[:,1]
hist_preg, x_edges_preg, y_edges_preg = np.histogram2d(phys_actions_iv_preg, phys_actions_vaso_preg, bins=5)

phys_actions_iv_cont = phys_actions_tuple_cont[:,0]
phys_actions_vaso_cont = phys_actions_tuple_cont[:,1]
hist_cont, x_edges_cont, y_edges_cont = np.histogram2d(phys_actions_iv_cont, phys_actions_vaso_cont, bins=5)

deeprl2_actions_iv_preg = deeprl2_actions_tuple_preg[:,0]
deeprl2_actions_vaso_preg = deeprl2_actions_tuple_preg[:,1]
hist2_preg, _, _ = np.histogram2d(deeprl2_actions_iv_preg, deeprl2_actions_vaso_preg, bins=5)

deeprl2_actions_iv_cont = deeprl2_actions_tuple_cont[:,0]
deeprl2_actions_vaso_cont = deeprl2_actions_tuple_cont[:,1]
hist2_cont, _, _ = np.histogram2d(deeprl2_actions_iv_cont, deeprl2_actions_vaso_cont, bins=5)

In [ ]:
x_edges_preg = np.arange(-0.5,5)
y_edges_preg = np.arange(-0.5,5)

x_edges_cont = np.arange(-0.5,5)
y_edges_cont = np.arange(-0.5,5)

In [ ]:
f, ((ax1, ax3), (ax2, ax4)) = plt.subplots(2, 2, figsize=(16,8))
ax1.imshow(np.flipud(hist_preg), cmap="Blues",extent=[x_edges_preg[0], x_edges_preg[-1],  y_edges_preg[0],y_edges_preg[-1]])
ax2.imshow(np.flipud(hist2_preg), cmap="Blues", extent=[x_edges_preg[0], x_edges_preg[-1],  y_edges_preg[0],y_edges_preg[-1]])
ax3.imshow(np.flipud(hist_cont), cmap="Blues",extent=[x_edges_cont[0], x_edges_cont[-1],  y_edges_cont[0],y_edges_cont[-1]])
ax4.imshow(np.flipud(hist2_cont), cmap="Blues", extent=[x_edges_cont[0], x_edges_cont[-1],  y_edges_cont[0],y_edges_cont[-1]])


# Major ticks
ax1.set_xticks(np.arange(0, 5, 1));
ax1.set_yticks(np.arange(0, 5, 1));
ax2.set_xticks(np.arange(0, 5, 1));
ax2.set_yticks(np.arange(0, 5, 1));
ax3.set_xticks(np.arange(0, 5, 1));
ax3.set_yticks(np.arange(0, 5, 1));
ax4.set_xticks(np.arange(0, 5, 1));
ax4.set_yticks(np.arange(0, 5, 1));

# Labels for major ticks
ax1.set_xticklabels(np.arange(0, 5, 1), fontsize=12);
ax1.set_yticklabels(np.arange(0, 5, 1), fontsize=12);
ax2.set_xticklabels(np.arange(0, 5, 1), fontsize=12);
ax2.set_yticklabels(np.arange(0, 5, 1), fontsize=12);
ax3.set_xticklabels(np.arange(0, 5, 1), fontsize=12);
ax3.set_yticklabels(np.arange(0, 5, 1), fontsize=12);
ax4.set_xticklabels(np.arange(0, 5, 1), fontsize=12);
ax4.set_yticklabels(np.arange(0, 5, 1), fontsize=12);


# Minor ticks
ax1.set_xticks(np.arange(-.5, 5, 1), minor=True);
ax1.set_yticks(np.arange(-.5, 5, 1), minor=True);
ax2.set_xticks(np.arange(-.5, 5, 1), minor=True);
ax2.set_yticks(np.arange(-.5, 5, 1), minor=True);
ax3.set_xticks(np.arange(-.5, 5, 1), minor=True);
ax3.set_yticks(np.arange(-.5, 5, 1), minor=True);
ax4.set_xticks(np.arange(-.5, 5, 1), minor=True);
ax4.set_yticks(np.arange(-.5, 5, 1), minor=True);


# Gridlines based on minor ticks
ax1.grid(which='minor', color='b', linestyle='-', linewidth=1)
ax2.grid(which='minor', color='b', linestyle='-', linewidth=1)
ax3.grid(which='minor', color='b', linestyle='-', linewidth=1)
ax4.grid(which='minor', color='b', linestyle='-', linewidth=1)

im1 = ax1.pcolormesh(x_edges_preg, y_edges_preg, np.log(hist_preg), cmap='Blues')
f.colorbar(im1, ax=ax1, label = "Action counts")

im2 = ax2.pcolormesh(x_edges_preg, y_edges_preg, np.log(hist2_preg), cmap='Blues')
f.colorbar(im2, ax=ax2, label = "Action counts")

im3 = ax3.pcolormesh(x_edges_cont, y_edges_cont, np.log(hist_cont), cmap='Blues')
f.colorbar(im3, ax=ax3, label = "Action counts")

im4 = ax4.pcolormesh(x_edges_cont, y_edges_cont, np.log(hist2_cont), cmap='Blues')
f.colorbar(im4, ax=ax4, label = "Action counts")


ax1.set_ylabel('IV fluid dose', fontsize=14)
ax2.set_ylabel('IV fluid dose', fontsize=14)
ax3.set_ylabel('IV fluid dose', fontsize=14)
ax4.set_ylabel('IV fluid dose', fontsize=14)
ax1.set_xlabel('Vasopressor dose', fontsize=14)
ax2.set_xlabel('Vasopressor dose', fontsize=14)
ax3.set_xlabel('Vasopressor dose', fontsize=14)
ax4.set_xlabel('Vasopressor dose', fontsize=14)

ax1.set_title("Physician policy (Pregnancy)", fontsize=18)
ax2.set_title("Q-Network policy (Pregnancy)", fontsize=18)
ax3.set_title("Physician policy (Control-Age&Sex)", fontsize=18)
ax4.set_title("Q-Network policy (Control-Age&Sex)", fontsize=18)
plt.tight_layout()